#### Schema customization

You can customize the generated `$ref` JSON location: the definitions are always stored under the key `definitions`, but a specified prefix can be used for the references.

This is useful if you need to extend or modify the JSON Schema default definitions location. E.g. with OpenAPI.

In [1]:
import json
from typing import Dict, Any, Type
from pydantic import BaseModel
from pydantic.schema import schema

In [2]:
class Foo(BaseModel):
    a: int

In [3]:
class Model(BaseModel):
    a: Foo

In [4]:
# Default location for OpenAPI
top_level_schema = schema([Model], ref_prefix='#/components/schemas/')
print(json.dumps(top_level_schema, indent=2))

{
  "definitions": {
    "Foo": {
      "title": "Foo",
      "type": "object",
      "properties": {
        "a": {
          "title": "A",
          "type": "integer"
        }
      },
      "required": [
        "a"
      ]
    },
    "Model": {
      "title": "Model",
      "type": "object",
      "properties": {
        "a": {
          "$ref": "#/components/schemas/Foo"
        }
      },
      "required": [
        "a"
      ]
    }
  }
}


It's also possible to extend/override the generated JSON schema in a model. To do it, use the `Config` sub-class attribute `schema_extra`. For example, you could add `examples` to the JSON Schema.

In [5]:
class Person(BaseModel):
    name: str
    age: int

    class Config:
        schema_extra = {
            'examples': [
                {
                    'name': 'John Doe',
                    'age': 25,
                }
            ]
        }

In [6]:
print(Person.schema_json(indent=2))

{
  "title": "Person",
  "type": "object",
  "properties": {
    "name": {
      "title": "Name",
      "type": "string"
    },
    "age": {
      "title": "Age",
      "type": "integer"
    }
  },
  "required": [
    "name",
    "age"
  ],
  "examples": [
    {
      "name": "John Doe",
      "age": 25
    }
  ]
}


For more fine-grained control, you can alternatively set `schema_extra` to a callable and post-process the generated schema. The callable can have one or two positional arguments. The first will be the schema dictionary. The second, if accepted, will be the model class. The callable is expected to mutate the schema dictionary in-place; the return value is not used.

For example, the `title` key can be removed from the model's `properties`:

In [7]:
class ComplexPerson(BaseModel):
    name: str
    age: int

    class Config:
        @staticmethod
        def schema_extra(schema: Dict[str, Any], model: Type['Person']) -> None:
            for prop in schema.get('properties', {}).values():
                prop.pop('title', None)

In [8]:
print(ComplexPerson.schema_json(indent=2))

{
  "title": "ComplexPerson",
  "type": "object",
  "properties": {
    "name": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    }
  },
  "required": [
    "name",
    "age"
  ]
}
